In [1]:
## In this pricer, we valorize the price of autocall at the start_date

In [2]:
#On importe les modeles qu'on a besoin:
import time
import datetime
from math import sqrt, exp, log
from random import gauss
import scipy.stats
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

In [3]:
tabDate = pd.read_csv('date.csv', sep= ";", infer_datetime_format= True)
date= [datetime.datetime.strptime(x, '%d/%m/%Y') for x in list(tabDate.date)]
start_date = date[0]
end_date = date[-1]

In [4]:
from pandas.tseries.holiday import USFederalHolidayCalendar
from datetime import timedelta

cal = USFederalHolidayCalendar()
holidays = pd.to_datetime(cal.holidays(start=start_date, end=end_date))

def daterange(date1, date2):
    date_range = [date1]
    for n in range(int ((date2 - date1).days)+1):
        date_next =  date1 + timedelta(n)
        if (date_next not in holidays) and (date_next.weekday() < 5):
            date_range.append(date_next)
    return date_range

In [5]:
l = len(date)
date_range= daterange(start_date, end_date)
dateIndicator= []
for i in range(l):
    dateIndicator.append(date_range.index(date[i]))
dateIndicator

[0, 252, 503, 753, 1004, 1255, 1506]

In [6]:
def autoCallPricer(S0,r,v,div,M,N,
                   coupon_barrier,kickout_barrier,protection_barrier,coupon_rate,notional, 
                   callable_clause = False):
    
    # N= date_range
    drift = r - div - 0.5*(v*v);
    dt= 1/252
    dx = v *sqrt(dt)
    values_matrix = np.zeros((2*M +1, N ))# value_matrix is the matrix of autocall value at each time and each asset price
    asset_prices = np.zeros(2*M +1) # We have 2M steps of asset prices

    # We calculate the parameters which we will use to calculate call price by backward deduction
    pu = (v*v*dt)/(2*dx*dx) + (drift*dt)/(2*dx)
    pm = 1.0 - (v*v*dt)/(dx*dx)
    pd = (v*v*dt)/(2*dx*dx) - (drift*dt)/(2*dx)
    for i in range(2 *M + 1):
        asset_prices[i] = S0 * exp((i- M) * dx)
    
    #Set the payoff at the maturity
    for i in range (2*M +1):
        if asset_prices[i]/S0 > coupon_barrier:
            values_matrix[i,-1] = notional + notional * coupon_rate
        elif asset_prices[i]/S0 > protection_barrier:
            values_matrix[i,-1] = notional
        else:
            values_matrix[i,-1] = notional * asset_prices[i]/S0
    
    # We set the boundary conditions: 
    
    ##Set lower boundary:
    for i in range(1,N):
        values_matrix[-1,-i -1] = values_matrix[-1,-i] * exp( -r * dt)
    ## Set ceiling boundary:
    for i in np.arange(len(dateIndicator) - 1, 0,-1):
        for j in np.arange(dateIndicator[i] -1, dateIndicator[i -1],-1):
            values_matrix[0,j] = values_matrix[0,j +1] * exp(-r * dt)
        values_matrix[0,dateIndicator[i -1]] = notional + notional * coupon_rate
   
    ##explicite difference implemention:
    for i in np.arange(len(dateIndicator) - 1, 0,-1):
        for j in np.arange(dateIndicator[i] -1, dateIndicator[i -1] -1,-1):
            for m in np.arange(2 * M -1, 0, -1):
                values_matrix[m][j] = exp( -r * dt)*(pu*values_matrix[m+1][j+1] + pm*values_matrix[m][j +1] + pd*values_matrix[m-1][j+1])
    
        if dateIndicator[i -1] > 0:
            for m in np.arange(2 * M -1, 0, -1):   
                if asset_prices[m]/S0 > kickout_barrier:
                    if callable_clause == False:
                        # The autocall is called
                        values_matrix[m,dateIndicator[i -1]] = notional + notional * coupon_rate
                    else:
                        # Investor has the choice betwwen exercising the product or keep the product
                        values_matrix[m,dateIndicator[i -1]] = notional * coupon_rate+ max(notional,values_matrix[m][dateIndicator[i -1]])
                elif asset_prices[m]/S0 > coupon_barrier:
                    values_matrix[m][dateIndicator[i -1]] += notional * coupon_rate # because the investor will receive the coupon
                else:
                    values_matrix[m][dateIndicator[i -1]] = values_matrix[m][j] 
    return values_matrix[M,0]
    

In [7]:
r= 0.04
v = 0.2
div= 0.00
S0 = 100 
N = len(date_range) # number of time steps
M = 500
coupon_barrier = 0.8
kickout_barrier = 1.1
protection_barrier = 0.6
coupon_rate = 0.088
notional = 1000

In [8]:
# The price of auto-call at the start_date:
autoCallPricer(S0,r,v,div,M,N,
               coupon_barrier,kickout_barrier,protection_barrier,coupon_rate,notional, 
               callable_clause = False)

1023.6468085093722

In [9]:
# The price of autocall with callable possibility at the start_date:
autoCallPricer(S0,r,v,div,M,N,
               coupon_barrier,kickout_barrier,protection_barrier,coupon_rate,notional, 
               callable_clause = True)


1116.983398381437